In [1]:
import math
from simtk import unit
import os
import tempfile
import pickle
import mdtraj as md
import numpy as np
from simtk.unit.quantity import Quantity
import logging

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

from matplotlib import pyplot as plt
from simtk.openmm import app
from tqdm import tqdm
import argparse
import random

In [20]:
def get_dihedrals_all_replicas(i, aa, length, out_dir, htf, dihedral_indices_new, dihedral_indices_old):
    new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
    old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)

    # From Hannah: https://github.com/hannahbrucemacdonald/endstate_pdbs/blob/master/scripts/input_for_pol_calc.py
    from perses.analysis.utils import open_netcdf
    nc = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{aa}_{length}ns.nc"))
    nc_checkpoint = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{aa}_{length}ns_checkpoint.nc"))
    checkpoint_interval = nc_checkpoint.CheckpointInterval
    all_positions = nc_checkpoint.variables['positions']
    n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)
    print(n_iter)
    n_iter = 1000
    
    from tqdm import tqdm
    dihedrals_master = []
    all_pos_hybrid_master = []
    for j in [0]:
        index = j # of replica
        all_pos_new = np.zeros(shape=(n_iter, new_top.n_atoms, 3))
        all_pos_old = np.zeros(shape=(n_iter, old_top.n_atoms, 3))
        all_pos_hybrid = np.zeros(shape=(n_iter, n_atoms, 3))
        for iteration in tqdm(range(0, n_iter)):
            replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == index)[0]
            pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
            all_pos_new[iteration] = htf.new_positions(pos).value_in_unit_system(unit.md_unit_system) # Get new positions only
            all_pos_hybrid[iteration] = pos # Get hybrid positions
            all_pos_old[iteration] = htf.old_positions(pos).value_in_unit_system(unit.md_unit_system)
        
            traj = md.Trajectory(np.array(all_pos_old[iteration]), old_top)
            dihedrals = md.compute_dihedrals(traj, np.array([dihedral_indices_old]))
            print(dihedrals)
            
        all_pos_hybrid_master.append(all_pos_hybrid)
        
        dihedrals_all = []
        for pos, top, indices in zip([all_pos_new, all_pos_old], [new_top, old_top], [dihedral_indices_new, dihedral_indices_old]):
            traj = md.Trajectory(np.array(pos), top)
    #         dihedrals = np.sin(md.compute_dihedrals(traj, np.array([indices])))
            dihedrals = md.compute_dihedrals(traj, np.array([indices]))
            dihedrals_all.append(dihedrals)
#         traj = md.Trajectory(np.array(all_pos_old), old_top)
#         dihedrals = md.compute_dihedrals(traj, np.array([dihedral_indices_old]))
#         dihedrals_all.append(dihedrals)
#         dihedrals_master.append(dihedrals_all)
    return dihedrals_all, n_iter, all_pos_hybrid_master

def plot_dihedrals(dihedrals, outfile):
    # Plot histogram with error bars : https://stackoverflow.com/questions/35390276/how-to-add-error-bars-to-histogram-diagram-in-python
    entries, edges, _ = plt.hist(dihedrals)
    bin_centers = 0.5 * (edges[:-1] + edges[1:]) # calculate bin centers
    plt.errorbar(bin_centers, entries, yerr=np.sqrt(entries), fmt='r.') # draw errobars, use the sqrt error
    plt.xlim(-np.pi, np.pi)
#     plt.savefig(outfile, dpi=300)
    plt.close()

def plot_time_series(dihedrals, n_iter, outfile):
    from perses.dispersed import feptasks
    t0, g, neff_max, a_t, uncorrelated_indices = feptasks.compute_timeseries(dihedrals)

    plt.scatter(range(n_iter), dihedrals)
    plt.ylabel("dihedral")
    plt.xlabel("iteration number")
    plt.ylim(-np.pi, np.pi)
    plt.show()
#     plt.savefig(outfile, dpi=300)
    plt.close()

    return uncorrelated_indices

def plot_dihedrals_uncorrelated(dihedrals, uncorrelated_indices, outfile):
    # Plot histogram with error bars : https://stackoverflow.com/questions/35390276/how-to-add-error-bars-to-histogram-diagram-in-python
    entries, edges, _ = plt.hist(dihedrals[uncorrelated_indices])
    bin_centers = 0.5 * (edges[:-1] + edges[1:]) # calculate bin centers
    plt.errorbar(bin_centers, entries, yerr=np.sqrt(entries), fmt='r.') # draw errobars, use the sqrt error
    plt.xlim(-np.pi, np.pi)
    plt.savefig(outfile, dpi=300)
    plt.close()



In [22]:
outdir = "/data/chodera/zhangi/perses_benchmark/myb/0/7/"
name = "LEU"
endstate = 1
phase = "apo"
length = 1
i = os.path.basename(os.path.dirname(outdir))
aa = name.lower()


In [23]:
with open(os.path.join(outdir, f"{i}_{phase}_{endstate}.pickle"), 'rb') as f:
    htf = pickle.load(f)

# if name == 'THR': # r-htf
#     indices_old = [669, 670, 673, 674]
#     indices_new = [669, 670, 673, 676]
# elif name == 'SER': # r-htf
#     indices_old = [11, 10, 7, 6]
#     indices_new = [15, 10, 7, 6]
# elif name == 'ALA': # r-htf
#     indices_old = [669, 670, 673, 674]
#     #indices_new = [0, 6, 7, 10] # phi angle
#     indices_new = [669, 670, 673, 676]
    

In [25]:
for atom in htf._topology_proposal.new_topology.atoms():
    print(atom, atom.residue.id)

<Atom 0 (C) of chain 0 residue 0 (ACE)> 89
<Atom 1 (O) of chain 0 residue 0 (ACE)> 89
<Atom 2 (CH3) of chain 0 residue 0 (ACE)> 89
<Atom 3 (H1) of chain 0 residue 0 (ACE)> 89
<Atom 4 (H2) of chain 0 residue 0 (ACE)> 89
<Atom 5 (H3) of chain 0 residue 0 (ACE)> 89
<Atom 6 (N) of chain 0 residue 1 (LYS)> 90
<Atom 7 (CA) of chain 0 residue 1 (LYS)> 90
<Atom 8 (C) of chain 0 residue 1 (LYS)> 90
<Atom 9 (O) of chain 0 residue 1 (LYS)> 90
<Atom 10 (CB) of chain 0 residue 1 (LYS)> 90
<Atom 11 (CG) of chain 0 residue 1 (LYS)> 90
<Atom 12 (CD) of chain 0 residue 1 (LYS)> 90
<Atom 13 (CE) of chain 0 residue 1 (LYS)> 90
<Atom 14 (NZ) of chain 0 residue 1 (LYS)> 90
<Atom 15 (H) of chain 0 residue 1 (LYS)> 90
<Atom 16 (HA) of chain 0 residue 1 (LYS)> 90
<Atom 17 (HB2) of chain 0 residue 1 (LYS)> 90
<Atom 18 (HB3) of chain 0 residue 1 (LYS)> 90
<Atom 19 (HG2) of chain 0 residue 1 (LYS)> 90
<Atom 20 (HG3) of chain 0 residue 1 (LYS)> 90
<Atom 21 (HD2) of chain 0 residue 1 (LYS)> 90
<Atom 22 (HD3) of ch

<Atom 942 (H1) of chain 1 residue 223 (HOH)> 225
<Atom 943 (H2) of chain 1 residue 223 (HOH)> 225
<Atom 944 (O) of chain 1 residue 224 (HOH)> 226
<Atom 945 (H1) of chain 1 residue 224 (HOH)> 226
<Atom 946 (H2) of chain 1 residue 224 (HOH)> 226
<Atom 947 (O) of chain 1 residue 225 (HOH)> 227
<Atom 948 (H1) of chain 1 residue 225 (HOH)> 227
<Atom 949 (H2) of chain 1 residue 225 (HOH)> 227
<Atom 950 (O) of chain 1 residue 226 (HOH)> 228
<Atom 951 (H1) of chain 1 residue 226 (HOH)> 228
<Atom 952 (H2) of chain 1 residue 226 (HOH)> 228
<Atom 953 (O) of chain 1 residue 227 (HOH)> 229
<Atom 954 (H1) of chain 1 residue 227 (HOH)> 229
<Atom 955 (H2) of chain 1 residue 227 (HOH)> 229
<Atom 956 (O) of chain 1 residue 228 (HOH)> 230
<Atom 957 (H1) of chain 1 residue 228 (HOH)> 230
<Atom 958 (H2) of chain 1 residue 228 (HOH)> 230
<Atom 959 (O) of chain 1 residue 229 (HOH)> 231
<Atom 960 (H1) of chain 1 residue 229 (HOH)> 231
<Atom 961 (H2) of chain 1 residue 229 (HOH)> 231
<Atom 962 (O) of chain 1 r

<Atom 1692 (H1) of chain 1 residue 473 (HOH)> 476
<Atom 1693 (H2) of chain 1 residue 473 (HOH)> 476
<Atom 1694 (O) of chain 1 residue 474 (HOH)> 477
<Atom 1695 (H1) of chain 1 residue 474 (HOH)> 477
<Atom 1696 (H2) of chain 1 residue 474 (HOH)> 477
<Atom 1697 (O) of chain 1 residue 475 (HOH)> 478
<Atom 1698 (H1) of chain 1 residue 475 (HOH)> 478
<Atom 1699 (H2) of chain 1 residue 475 (HOH)> 478
<Atom 1700 (O) of chain 1 residue 476 (HOH)> 479
<Atom 1701 (H1) of chain 1 residue 476 (HOH)> 479
<Atom 1702 (H2) of chain 1 residue 476 (HOH)> 479
<Atom 1703 (O) of chain 1 residue 477 (HOH)> 480
<Atom 1704 (H1) of chain 1 residue 477 (HOH)> 480
<Atom 1705 (H2) of chain 1 residue 477 (HOH)> 480
<Atom 1706 (O) of chain 1 residue 478 (HOH)> 481
<Atom 1707 (H1) of chain 1 residue 478 (HOH)> 481
<Atom 1708 (H2) of chain 1 residue 478 (HOH)> 481
<Atom 1709 (O) of chain 1 residue 479 (HOH)> 482
<Atom 1710 (H1) of chain 1 residue 479 (HOH)> 482
<Atom 1711 (H2) of chain 1 residue 479 (HOH)> 482
<Atom 

<Atom 2442 (H1) of chain 1 residue 723 (HOH)> 727
<Atom 2443 (H2) of chain 1 residue 723 (HOH)> 727
<Atom 2444 (O) of chain 1 residue 724 (HOH)> 728
<Atom 2445 (H1) of chain 1 residue 724 (HOH)> 728
<Atom 2446 (H2) of chain 1 residue 724 (HOH)> 728
<Atom 2447 (O) of chain 1 residue 725 (HOH)> 729
<Atom 2448 (H1) of chain 1 residue 725 (HOH)> 729
<Atom 2449 (H2) of chain 1 residue 725 (HOH)> 729
<Atom 2450 (O) of chain 1 residue 726 (HOH)> 730
<Atom 2451 (H1) of chain 1 residue 726 (HOH)> 730
<Atom 2452 (H2) of chain 1 residue 726 (HOH)> 730
<Atom 2453 (O) of chain 1 residue 727 (HOH)> 731
<Atom 2454 (H1) of chain 1 residue 727 (HOH)> 731
<Atom 2455 (H2) of chain 1 residue 727 (HOH)> 731
<Atom 2456 (O) of chain 1 residue 728 (HOH)> 732
<Atom 2457 (H1) of chain 1 residue 728 (HOH)> 732
<Atom 2458 (H2) of chain 1 residue 728 (HOH)> 732
<Atom 2459 (O) of chain 1 residue 729 (HOH)> 733
<Atom 2460 (H1) of chain 1 residue 729 (HOH)> 733
<Atom 2461 (H2) of chain 1 residue 729 (HOH)> 733
<Atom 

<Atom 3191 (O) of chain 1 residue 973 (HOH)> 978
<Atom 3192 (H1) of chain 1 residue 973 (HOH)> 978
<Atom 3193 (H2) of chain 1 residue 973 (HOH)> 978
<Atom 3194 (O) of chain 1 residue 974 (HOH)> 979
<Atom 3195 (H1) of chain 1 residue 974 (HOH)> 979
<Atom 3196 (H2) of chain 1 residue 974 (HOH)> 979
<Atom 3197 (O) of chain 1 residue 975 (HOH)> 980
<Atom 3198 (H1) of chain 1 residue 975 (HOH)> 980
<Atom 3199 (H2) of chain 1 residue 975 (HOH)> 980
<Atom 3200 (O) of chain 1 residue 976 (HOH)> 981
<Atom 3201 (H1) of chain 1 residue 976 (HOH)> 981
<Atom 3202 (H2) of chain 1 residue 976 (HOH)> 981
<Atom 3203 (O) of chain 1 residue 977 (HOH)> 982
<Atom 3204 (H1) of chain 1 residue 977 (HOH)> 982
<Atom 3205 (H2) of chain 1 residue 977 (HOH)> 982
<Atom 3206 (O) of chain 1 residue 978 (HOH)> 983
<Atom 3207 (H1) of chain 1 residue 978 (HOH)> 983
<Atom 3208 (H2) of chain 1 residue 978 (HOH)> 983
<Atom 3209 (O) of chain 1 residue 979 (HOH)> 984
<Atom 3210 (H1) of chain 1 residue 979 (HOH)> 984
<Atom 3

...

<Atom 5441 (O) of chain 1 residue 1723 (HOH)> 1732
<Atom 5442 (H1) of chain 1 residue 1723 (HOH)> 1732
<Atom 5443 (H2) of chain 1 residue 1723 (HOH)> 1732
<Atom 5444 (O) of chain 1 residue 1724 (HOH)> 1733
<Atom 5445 (H1) of chain 1 residue 1724 (HOH)> 1733
<Atom 5446 (H2) of chain 1 residue 1724 (HOH)> 1733
<Atom 5447 (O) of chain 1 residue 1725 (HOH)> 1734
<Atom 5448 (H1) of chain 1 residue 1725 (HOH)> 1734
<Atom 5449 (H2) of chain 1 residue 1725 (HOH)> 1734
<Atom 5450 (O) of chain 1 residue 1726 (HOH)> 1735
<Atom 5451 (H1) of chain 1 residue 1726 (HOH)> 1735
<Atom 5452 (H2) of chain 1 residue 1726 (HOH)> 1735
<Atom 5453 (O) of chain 1 residue 1727 (HOH)> 1736
<Atom 5454 (H1) of chain 1 residue 1727 (HOH)> 1736
<Atom 5455 (H2) of chain 1 residue 1727 (HOH)> 1736
<Atom 5456 (O) of chain 1 residue 1728 (HOH)> 1737
<Atom 5457 (H1) of chain 1 residue 1728 (HOH)> 1737
<Atom 5458 (H2) of chain 1 residue 1728 (HOH)> 1737
<Atom 5459 (O) of chain 1 residue 1729 (HOH)> 1738
<Atom 5460 (H1) of 

<Atom 6190 (H2) of chain 1 residue 1972 (HOH)> 1983
<Atom 6191 (O) of chain 1 residue 1973 (HOH)> 1984
<Atom 6192 (H1) of chain 1 residue 1973 (HOH)> 1984
<Atom 6193 (H2) of chain 1 residue 1973 (HOH)> 1984
<Atom 6194 (O) of chain 1 residue 1974 (HOH)> 1985
<Atom 6195 (H1) of chain 1 residue 1974 (HOH)> 1985
<Atom 6196 (H2) of chain 1 residue 1974 (HOH)> 1985
<Atom 6197 (O) of chain 1 residue 1975 (HOH)> 1986
<Atom 6198 (H1) of chain 1 residue 1975 (HOH)> 1986
<Atom 6199 (H2) of chain 1 residue 1975 (HOH)> 1986
<Atom 6200 (O) of chain 1 residue 1976 (HOH)> 1987
<Atom 6201 (H1) of chain 1 residue 1976 (HOH)> 1987
<Atom 6202 (H2) of chain 1 residue 1976 (HOH)> 1987
<Atom 6203 (O) of chain 1 residue 1977 (HOH)> 1988
<Atom 6204 (H1) of chain 1 residue 1977 (HOH)> 1988
<Atom 6205 (H2) of chain 1 residue 1977 (HOH)> 1988
<Atom 6206 (O) of chain 1 residue 1978 (HOH)> 1989
<Atom 6207 (H1) of chain 1 residue 1978 (HOH)> 1989
<Atom 6208 (H2) of chain 1 residue 1978 (HOH)> 1989
<Atom 6209 (O) of 

<Atom 6940 (H2) of chain 1 residue 2222 (HOH)> 2235
<Atom 6941 (O) of chain 1 residue 2223 (HOH)> 2236
<Atom 6942 (H1) of chain 1 residue 2223 (HOH)> 2236
<Atom 6943 (H2) of chain 1 residue 2223 (HOH)> 2236
<Atom 6944 (O) of chain 1 residue 2224 (HOH)> 2237
<Atom 6945 (H1) of chain 1 residue 2224 (HOH)> 2237
<Atom 6946 (H2) of chain 1 residue 2224 (HOH)> 2237
<Atom 6947 (O) of chain 1 residue 2225 (HOH)> 2238
<Atom 6948 (H1) of chain 1 residue 2225 (HOH)> 2238
<Atom 6949 (H2) of chain 1 residue 2225 (HOH)> 2238
<Atom 6950 (O) of chain 1 residue 2226 (HOH)> 2239
<Atom 6951 (H1) of chain 1 residue 2226 (HOH)> 2239
<Atom 6952 (H2) of chain 1 residue 2226 (HOH)> 2239
<Atom 6953 (O) of chain 1 residue 2227 (HOH)> 2240
<Atom 6954 (H1) of chain 1 residue 2227 (HOH)> 2240
<Atom 6955 (H2) of chain 1 residue 2227 (HOH)> 2240
<Atom 6956 (O) of chain 1 residue 2228 (HOH)> 2241
<Atom 6957 (H1) of chain 1 residue 2228 (HOH)> 2241
<Atom 6958 (H2) of chain 1 residue 2228 (HOH)> 2241
<Atom 6959 (O) of 

<Atom 7690 (H2) of chain 1 residue 2472 (HOH)> 2487
<Atom 7691 (O) of chain 1 residue 2473 (HOH)> 2488
<Atom 7692 (H1) of chain 1 residue 2473 (HOH)> 2488
<Atom 7693 (H2) of chain 1 residue 2473 (HOH)> 2488
<Atom 7694 (O) of chain 1 residue 2474 (HOH)> 2489
<Atom 7695 (H1) of chain 1 residue 2474 (HOH)> 2489
<Atom 7696 (H2) of chain 1 residue 2474 (HOH)> 2489
<Atom 7697 (O) of chain 1 residue 2475 (HOH)> 2490
<Atom 7698 (H1) of chain 1 residue 2475 (HOH)> 2490
<Atom 7699 (H2) of chain 1 residue 2475 (HOH)> 2490
<Atom 7700 (O) of chain 1 residue 2476 (HOH)> 2492
<Atom 7701 (H1) of chain 1 residue 2476 (HOH)> 2492
<Atom 7702 (H2) of chain 1 residue 2476 (HOH)> 2492
<Atom 7703 (O) of chain 1 residue 2477 (HOH)> 2493
<Atom 7704 (H1) of chain 1 residue 2477 (HOH)> 2493
<Atom 7705 (H2) of chain 1 residue 2477 (HOH)> 2493
<Atom 7706 (O) of chain 1 residue 2478 (HOH)> 2494
<Atom 7707 (H1) of chain 1 residue 2478 (HOH)> 2494
<Atom 7708 (H2) of chain 1 residue 2478 (HOH)> 2494
<Atom 7709 (O) of 

<Atom 8440 (H2) of chain 1 residue 2722 (HOH)> 2739
<Atom 8441 (O) of chain 1 residue 2723 (HOH)> 2740
<Atom 8442 (H1) of chain 1 residue 2723 (HOH)> 2740
<Atom 8443 (H2) of chain 1 residue 2723 (HOH)> 2740
<Atom 8444 (O) of chain 1 residue 2724 (HOH)> 2741
<Atom 8445 (H1) of chain 1 residue 2724 (HOH)> 2741
<Atom 8446 (H2) of chain 1 residue 2724 (HOH)> 2741
<Atom 8447 (O) of chain 1 residue 2725 (HOH)> 2742
<Atom 8448 (H1) of chain 1 residue 2725 (HOH)> 2742
<Atom 8449 (H2) of chain 1 residue 2725 (HOH)> 2742
<Atom 8450 (O) of chain 1 residue 2726 (HOH)> 2743
<Atom 8451 (H1) of chain 1 residue 2726 (HOH)> 2743
<Atom 8452 (H2) of chain 1 residue 2726 (HOH)> 2743
<Atom 8453 (O) of chain 1 residue 2727 (HOH)> 2744
<Atom 8454 (H1) of chain 1 residue 2727 (HOH)> 2744
<Atom 8455 (H2) of chain 1 residue 2727 (HOH)> 2744
<Atom 8456 (O) of chain 1 residue 2728 (HOH)> 2745
<Atom 8457 (H1) of chain 1 residue 2728 (HOH)> 2745
<Atom 8458 (H2) of chain 1 residue 2728 (HOH)> 2745
<Atom 8459 (O) of 

In [18]:
# thr_dihedral = ['N', 'CA', 'CB', 'OG']
# ring_dihedral = ['N', 'CA', 'CB', 'CG']
# ala_dihedral = ['N', 'CA', 'CB', 'HB1']

# dihedral_atoms = []
# for aa_name in ["TYR", "PHE"]:
#     if aa_name in ["PHE", "TYR", "TRP"]:
#         dihedral_atoms.append(ring_dihedral)
#     elif aa_name == "THR":
#         dihedral_atoms.append(thr_dihedral)
#     elif aa_name == "ALA":
#         dihedral_atoms.append(ala_dihedral)

# indices_old = [atom.index for atom in list(htf._topology_proposal.old_topology.residues())[int("99")].atoms() if atom.name in dihedral_atoms[0]]
# indices_new = [atom.index for atom in list(htf._topology_proposal.new_topology.residues())[int("99")].atoms() if atom.name in dihedral_atoms[1]]

indices_old = [180, 181, 184, 185]
indices_new = []

In [16]:
indices_old

[180, 181, 184, 185]

In [21]:
dihedrals, n_iter, all_pos_hybrid = get_dihedrals_all_replicas(i, aa, length, outdir, htf, indices_new, indices_old)                                       


  0%|          | 0/1000 [00:00<?, ?it/s]

292


  0%|          | 1/1000 [00:04<1:12:05,  4.33s/it]

[[-2.965972]]


  0%|          | 2/1000 [00:08<1:12:08,  4.34s/it]

...

[[2.9530377]]


  4%|▍         | 44/1000 [03:16<1:16:03,  4.77s/it]

[[-2.830808]]


  4%|▍         | 44/1000 [03:19<1:12:04,  4.52s/it]


KeyboardInterrupt: 

In [ ]:
new_aa = "gly"
old_aa = "leu"

# for j, replica in tqdm(enumerate(dihedrals)):
dihedrals_new = dihedrals[0]
dihedrals_old = dihedrals[1]
#     dihedrals_old = replica[0]
plot_dihedrals(dihedrals_old, os.path.join(outdir, f"{i}_{phase}_{aa}_{length}_{old_aa}_correlated.png"))
uncorrelated_old = plot_time_series(dihedrals_old, n_iter, os.path.join(outdir, f"{i}_{phase}_{aa}_{length}_{old_aa}_timeseries.png"))
#     plot_dihedrals_uncorrelated(dihedrals_old, uncorrelated_old, os.path.join(out_dir, f"{i}_{args.phase}_{aa}_{length}_{j}_{old_aa}_decorrelated.png"))
plot_dihedrals(dihedrals_new.flatten(), os.path.join(outdir, f"{i}_{phase}_{aa}_{length}_{new_aa}_correlated.png"))
uncorrelated_new = plot_time_series(dihedrals_new, n_iter, os.path.join(outdir, f"{i}_{phase}_{aa}_{length}__{new_aa}_timeseries.png"))
#     plot_dihedrals_uncorrelated(dihedrals_new, uncorrelated_new, os.path.join(out_dir, f"{i}_{args.phase}_{aa}_{length}_{new_aa}_decorrelated.png"))

In [36]:
# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

In [11]:
temperatures

[Quantity(value=298.0, unit=kelvin),
 Quantity(value=347.95853028599566, unit=kelvin),
 Quantity(value=402.6715859162709, unit=kelvin),
 Quantity(value=462.5916524047109, unit=kelvin),
 Quantity(value=528.2142780581083, unit=kelvin),
 Quantity(value=600.0821722380443, unit=kelvin),
 Quantity(value=678.7896936520565, unit=kelvin),
 Quantity(value=764.9877657929634, unit=kelvin),
 Quantity(value=859.3892601777964, unit=kelvin),
 Quantity(value=962.7748919065651, unit=kelvin),
 Quantity(value=1075.999676298049, unit=kelvin),
 Quantity(value=1200.0, unit=kelvin)]

In [16]:
# Create states for each replica
n_replicas = 20  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 2000.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

In [17]:
temperatures

[Quantity(value=298.0, unit=kelvin),
 Quantity(value=351.5291669665935, unit=kelvin),
 Quantity(value=407.95111663096566, unit=kelvin),
 Quantity(value=467.42217856465265, unit=kelvin),
 Quantity(value=530.1071305834903, unit=kelvin),
 Quantity(value=596.1796553012331, unit=kelvin),
 Quantity(value=665.8228213559005, unit=kelvin),
 Quantity(value=739.2295906421895, unit=kelvin),
 Quantity(value=816.603352955359, unit=kelvin),
 Quantity(value=898.1584895279351, unit=kelvin),
 Quantity(value=984.1209670206374, unit=kelvin),
 Quantity(value=1074.7289636133153, unit=kelvin),
 Quantity(value=1170.233528930615, unit=kelvin),
 Quantity(value=1270.8992796308519, unit=kelvin),
 Quantity(value=1377.0051325853656, unit=kelvin),
 Quantity(value=1488.845077679805, unit=kelvin),
 Quantity(value=1606.7289923785465, unit=kelvin),
 Quantity(value=1730.983500309194, unit=kelvin),
 Quantity(value=1861.952876246047, unit=kelvin),
 Quantity(value=2000.0, unit=kelvin)]

In [13]:
# Create states for each replica
n_replicas = 80  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 10000.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

In [14]:
temperatures

[Quantity(value=298.0, unit=kelvin),
 Quantity(value=369.9269062163677, unit=kelvin),
 Quantity(value=442.770066441733, unit=kelvin),
 Quantity(value=516.5411525448463, unit=kelvin),
 Quantity(value=591.2519850786861, unit=kelvin),
 Quantity(value=666.9145351745013, unit=kelvin),
 Quantity(value=743.5409264599768, unit=kelvin),
 Quantity(value=821.1434370018429, unit=kelvin),
 Quantity(value=899.7345012732171, unit=kelvin),
 Quantity(value=979.3267121460236, unit=kelvin),
 Quantity(value=1059.9328229087791, unit=kelvin),
 Quantity(value=1141.5657493100928, unit=kelvin),
 Quantity(value=1224.2385716281906, unit=kelvin),
 Quantity(value=1307.9645367668102, unit=kelvin),
 Quantity(value=1392.7570603777847, unit=kelvin),
 Quantity(value=1478.6297290106777, unit=kelvin),
 Quantity(value=1565.5963022897897, unit=kelvin),
 Quantity(value=1653.670715118902, unit=kelvin),
 Quantity(value=1742.8670799141112, unit=kelvin),
 Quantity(value=1833.1996888651004, unit=kelvin),
 Quantity(value=1924.683